In [1]:
import struct
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os

In [2]:
cfg_file = "/home/curium/repos/DSVT/tools/cfgs/custom_models/dsvt_small.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_files')==False:
    os.mkdir('./deploy_files')
log_file = './deploy_files/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "/media/curium/volume2/weights/dsvt_small_4_50.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()

2025-06-16 15:15:31,408   INFO  Loading Custom dataset.
2025-06-16 15:15:31,414   INFO  Total samples for CUSTOM dataset: 100
2025-06-16 15:15:32,696   INFO  ==> Loading parameters from checkpoint /media/curium/volume2/weights/dsvt_small_4_50.pth to GPU
2025-06-16 15:15:32,748   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+3db84b0
2025-06-16 15:15:32,761   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicPillarVFE_3d(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=7, out_features=32, bias=False)
        (norm): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=64, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0): ModuleList(
            (0): PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=64, bias=True)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
                (3): 

In [3]:
with open('dsvt_small_4_50.wts', 'w') as f:
        f.write('{}\n'.format(len(model.state_dict().keys())))
        for k, v in model.state_dict().items():
            print('**'*10)
            print(k)
            # print(v)
            print(v.shape)
            # pdb.set_trace()
            vr = v.reshape(-1).cpu().numpy()
            f.write('{} {} '.format(k, len(vr)))
            for vv in vr:
                f.write(' ')
                f.write(struct.pack('>f', float(vv)).hex())
            f.write('\n')

********************
global_step
torch.Size([1])
********************
vfe.pfn_layers.0.linear.weight
torch.Size([32, 7])
********************
vfe.pfn_layers.0.norm.weight
torch.Size([32])
********************
vfe.pfn_layers.0.norm.bias
torch.Size([32])
********************
vfe.pfn_layers.0.norm.running_mean
torch.Size([32])
********************
vfe.pfn_layers.0.norm.running_var
torch.Size([32])
********************
vfe.pfn_layers.0.norm.num_batches_tracked
torch.Size([])
********************
vfe.pfn_layers.1.linear.weight
torch.Size([64, 64])
********************
vfe.pfn_layers.1.norm.weight
torch.Size([64])
********************
vfe.pfn_layers.1.norm.bias
torch.Size([64])
********************
vfe.pfn_layers.1.norm.running_mean
torch.Size([64])
********************
vfe.pfn_layers.1.norm.running_var
torch.Size([64])
********************
vfe.pfn_layers.1.norm.num_batches_tracked
torch.Size([])
********************
backbone_3d.input_layer.posembed_layers.0.0.0.position_embedding_head.0.wei